# CSV Clean
This Notebook contains functions to clean the electorate and election datasets and write the new files into CSVs in a clean_data folder

## Cleaning Functions

In [24]:
# import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from data_cleaning.country_match import country_code_map as geomap
from data_cleaning import cleaning_utilities
from data_cleaning.education_codes import add_education_codes
from data_cleaning.education_codes import clean2018

In [25]:
#Extract Dataframes and clean countries/education codes
def extract_dataframe(path: str, remove_extra_header=False):
    '''
    Creates a Pandas Data Frame from a CSV File and makes headers uniform (lower cases, separated by _ vs space)
    :param path - A path to a CSV File
    :param remove_extra_header - A boolean to indicate if there is an extra header (Portuguese) which can be removed
    :return a Pandas dataframe
    '''
    
    df = pd.DataFrame()
    
    if remove_extra_header:
        df = pd.read_csv(path, skiprows=[1])
        new_column_names = [x.lower().replace(' ','_') for x in df.columns]
        df.columns = new_column_names
        
    else:
        df = pd.read_csv(path)
        new_column_names = [x.lower().replace(' ','_') for x in df.columns]
        df.columns = new_column_names
    
    
    return df

In [26]:
def standardize_country_name(df):
    '''
    Standardizes Country Name (by removing accents)
    :param path - A pandas dataframe
    :return The updated pandas dataframe (with accents removed)
    '''
    #Remove accents from country names
    countries = df['municipality_name'].tolist()
    countries = [cleaning_utilities.remove_accents(country) for country in countries]
    df['municipality_name'] = countries
    
    
    return df

In [27]:
def standardize_edu_attainment(df):
    '''
    Standardizes Educational Attainment Description (by removing accents)
    :param path - A pandas dataframe
    :return The updated pandas dataframe (with accents removed)
    '''
    #Remove accents from educational attainment description
    education_levels = df["educational_attainment_description"].tolist()
    education_levels = [cleaning_utilities.remove_accents(edu_desc) for edu_desc in education_levels]
    df["educational_attainment_description"] = education_levels
    
    
    return df

In [28]:
#Standardize Country Code and Names across Dataset
def standardize_country_code(df_orig):
    '''
    Standardizes post-2010 municipality codes to pre-2010 country codes
    :param df_orig - A Pandas dataframe
    :return a Pandas dataframe with the municipality code transformation applied

    '''
    df = df_orig.copy() #Create a copy of the dataframe
    
    #Iterate through dataframe and update municipality code and name
    for row_idx in df.index:
        code = df.at[row_idx, 'municipality_code']
        new_code, name = geomap[code]
        df.at[row_idx, 'municipality_code'] = new_code
        df.at[row_idx, 'municipality_name'] = name
    
    return df

In [29]:
#Standardize Party Names (All Uppercase)
def standardize_party_name(df):
    '''
    Standardizes political party names so that they are ALL CAPS
    :param df - A Pandas dataframe
    :return a Pandas dataframe with the municipality code transformation applied

    '''
    
    #Remove Accents and make all Party names ALL CAPS
    parties = df["party_name"]
    parties = [cleaning_utilities.remove_accents(party).upper() for party in parties]
    df["party_name"] = parties
    
    return df

    

In [30]:
#Write to CSV
def write_to_csv(df, path:str):
    '''
    Writes Cleaned Data Files to CSV
    :param df - A Pandas dataframe
    :param path - the location the CSV should be written to 

    '''
    
    df.to_csv(path_or_buf=path)

## Clean Electorate Data

In [31]:
#Get Files

# obtain the data from csv
df_1998 = extract_dataframe('../data/original/perfil_eleitorado_1998.csv', remove_extra_header=True)
df_2002 = extract_dataframe('../data/original/perfil_eleitorado_2002.csv', remove_extra_header=True)
df_2006 = extract_dataframe('../data/original/perfil_eleitorado_2006.csv', remove_extra_header=True)
df_2010 = extract_dataframe('../data/original/perfil_eleitorado_2010.csv', remove_extra_header=False)
df_2014 = extract_dataframe('../data/original/perfil_eleitorado_2014.csv', remove_extra_header=True)
df_2018 = extract_dataframe('../data/original/perfil_eleitorado_2018.csv', remove_extra_header=True)

#Standardize Country Codes
df_2010 = standardize_country_code(df_2010)
df_2014 = standardize_country_code(df_2014)
df_2018 = standardize_country_code(df_2018)

#Standardize Country Names
df_1998 = standardize_country_name(df_1998)
df_2002 = standardize_country_name(df_2002)
df_2006 = standardize_country_name(df_2006)
df_2010 = standardize_country_name(df_2010)
df_2014 = standardize_country_name(df_2014)
df_2018 = standardize_country_name(df_2018)

#Standardize Educational Attainment Descriptions
df_1998 = standardize_edu_attainment(df_1998)
df_2002 = standardize_edu_attainment(df_2002)
df_2006 = standardize_edu_attainment(df_2006)
df_2010 = standardize_edu_attainment(df_2010)
df_2014 = standardize_edu_attainment(df_2014)
df_2018 = standardize_edu_attainment(df_2018)


# Adding educational attainment codes for 2010 and resolving the issue of 2018 accents
clean2018(df_2018)
df_2010 = add_education_codes(df_2002, df_2010)

In [32]:
#Write Clean Data To CSV
write_to_csv(df_1998, path='../data/clean/perfil_eleitorado_1998.csv')
write_to_csv(df_2002, path='../data/clean/perfil_eleitorado_2002.csv')
write_to_csv(df_2006, path='../data/clean/perfil_eleitorado_2006.csv')
write_to_csv(df_2010, path='../data/clean/perfil_eleitorado_2010.csv')
write_to_csv(df_2014, path='../data/clean/perfil_eleitorado_2014.csv')
write_to_csv(df_2018, path='../data/clean/perfil_eleitorado_2018.csv')

## Clean Election Data

In [33]:
#Extract DataFrames
df_1998 = extract_dataframe('../data/original/1998_Election_Data.csv', remove_extra_header=True)
df_2002 = extract_dataframe('../data/original/2002_Election_Data.csv', remove_extra_header=True)
df_2006 = extract_dataframe('../data/original/2006_Election_Data.csv', remove_extra_header=True)
df_2010 = extract_dataframe('../data/original/2010_Election_Data.csv', remove_extra_header=True)
df_2014 = extract_dataframe('../data/original/2014_Election_Data.csv', remove_extra_header=True)
df_2018 = extract_dataframe('../data/original/2018_Election_Data.csv', remove_extra_header=True)

#Standardize Country Codes
df_2010 = standardize_country_code(df_2010)
df_2014 = standardize_country_code(df_2014)
df_2018 = standardize_country_code(df_2018)

#Standardize Country Names
df_1998 = standardize_country_name(df_1998)
df_2002 = standardize_country_name(df_2002)
df_2006 = standardize_country_name(df_2006)
df_2010 = standardize_country_name(df_2010)
df_2014 = standardize_country_name(df_2014)
df_2018 = standardize_country_name(df_2018)

#Standardize Party Names
df_1998 = standardize_party_name(df_1998)
df_2002 = standardize_party_name(df_2002)
df_2006 = standardize_party_name(df_2006)
df_2010 = standardize_party_name(df_2010)
df_2014 = standardize_party_name(df_2014)
df_2018 = standardize_party_name(df_2018)


In [34]:
#Write Clean Data To CSV
write_to_csv(df_1998, path='../data/clean/1998_Election_Data.csv')
write_to_csv(df_2002, path='../data/clean/2002_Election_Data.csv')
write_to_csv(df_2006, path='../data/clean/2006_Election_Data.csv')
write_to_csv(df_2010, path='../data/clean/2010_Election_Data.csv')
write_to_csv(df_2014, path='../data/clean/2014_Election_Data.csv')
write_to_csv(df_2018, path='../data/clean/2018_Election_Data.csv')